<a href="https://colab.research.google.com/github/benitomartin/rag-qwen-vllm-milvus/blob/main/rag_qwen_vllm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Install and Import Dependencies

This notebook runs using a A100 GPU.

In [ ]:
!pip -qq install vllm triton langchain-community langchain-milvus langchain-openai

In [2]:
import os
import torch
import gc
from google.colab import userdata

from langchain_milvus import Milvus
from langchain_community.llms import VLLM
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Step 2: Set Environment

In [3]:
# Check the GPU
!nvidia-smi

Tue Dec 17 13:48:47 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0              46W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [4]:
# Run garbage collector
gc.collect()

# Clear the GPU memory cache.
torch.cuda.empty_cache()

In [5]:
# Set environment variable

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
# os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Step 3: Load the Data

In [6]:
# Load documents

loader = WebBaseLoader(
    web_paths=(
        "https://qwenlm.github.io/blog/qwq-32b-preview/",
        "https://qwenlm.github.io/blog/qwen2.5/",
        "https://qwenlm.github.io/blog/qwen2.5-coder-family/",

    )
)

documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)

docs = text_splitter.split_documents(documents)

In [7]:
docs[1]

Document(metadata={'source': 'https://qwenlm.github.io/blog/qwq-32b-preview/', 'title': 'QwQ: Reflect Deeply on the Boundaries of the Unknown | Qwen', 'description': 'GITHUB HUGGING FACE MODELSCOPE DEMO DISCORD\nNote: This is the pronunciation of QwQ: /kwju:/ , similar to the word “quill”.\nWhat does it mean to think, to question, to understand? These are the deep waters that QwQ (Qwen with Questions) wades into. Like an eternal student of wisdom, it approaches every problem - be it mathematics, code, or knowledge of our world - with genuine wonder and doubt. QwQ embodies that ancient philosophical spirit: it knows that it knows nothing, and that’s precisely what drives its curiosity.', 'language': 'en'}, page_content='ResourcesBlogPublicationAboutQwQ: Reflect Deeply on the Boundaries of the UnknownNovember 28, 2024\xa0·\xa022 min\xa0·\xa04496 words\xa0·\xa0Qwen Team\xa0|\xa0Translations:简体中文GITHUB\nHUGGING FACE\nMODELSCOPE\nDEMO')

# Step 4: Create Milvus Retriever

In [8]:
# Set Milvus retriever

embeddings = OpenAIEmbeddings()

vectorstore = Milvus.from_documents(
    documents=docs,
    embedding=embeddings,
    connection_args={
        "uri": "./milvus_demo.db",
    },
    text_field="page_content",
    metadata_field="metadata",
    drop_old=True,
)

retriever = vectorstore.as_retriever(
                          search_type="similarity",
                          search_kwargs={"k": 4}
                          )

DEBUG:pymilvus.milvus_client.milvus_client:Created new connection using: 07a4978bc2544501bc96f103fe3edb00


# Step 5: Initialize LLM Engine

In [9]:
# Initialize vLLM

llm = VLLM(
    model="Qwen/Qwen2.5-1.5B",
    max_new_tokens=500,
    enforce_eager=True,
    dtype="bfloat16",
    gpu_memory_utilization=0.8,
)

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

INFO 12-17 13:49:28 config.py:350] This model supports multiple tasks: {'generate', 'embedding'}. Defaulting to 'generate'.
WARNING 12-17 13:49:28 arg_utils.py:1013] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 12-17 13:49:28 config.py:1136] Chunked prefill is enabled with max_num_batched_tokens=512.
INFO 12-17 13:49:28 llm_engine.py:249] Initializing an LLM engine (v0.6.4.post1) with config: model='Qwen/Qwen2.5-1.5B', speculative_config=None, tokenizer='Qwen/Qwen2.5-1.5B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=131072, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=Fal

tokenizer_config.json:   0%|          | 0.00/7.23k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

...

INFO 12-17 13:49:40 selector.py:135] Using Flash Attention backend.
INFO 12-17 13:49:40 model_runner.py:1072] Starting to load model Qwen/Qwen2.5-1.5B...
INFO 12-17 13:49:41 weight_utils.py:243] Using model weights format ['*.safetensors']


model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

INFO 12-17 13:50:59 weight_utils.py:288] No model.safetensors.index.json found in remote.


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 12-17 13:51:00 model_runner.py:1077] Loading model weights took 2.9104 GB
INFO 12-17 13:51:01 worker.py:232] Memory profiling results: total_gpu_memory=39.56GiB initial_memory_usage=3.53GiB peak_torch_memory=4.30GiB memory_usage_post_profile=3.57GiB non_torch_memory=0.65GiB kv_cache_size=30.66GiB gpu_memory_utilization=0.90
INFO 12-17 13:51:01 gpu_executor.py:113] # GPU blocks: 71770, # CPU blocks: 9362
INFO 12-17 13:51:01 gpu_executor.py:117] Maximum concurrency for 131072 tokens per request: 8.76x
INFO 12-17 13:51:05 model_runner.py:1400] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 12-17 13:51:05 model_runner.py:1404] If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utilization` or switching to eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO

# Step 6: Implement RAG Pipeline


In [10]:
# Set QA chain

template = """
              You are an assistant for question-answering tasks.
              Use the following pieces of retrieved context to answer the question.
              If you don't know the answer, just say that you don't know.
              Pleae provide the answer in English language.

              Question: {question}
              Context: {context}

              Answer:
          """

prompt = PromptTemplate.from_template(template)

qa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever=retriever,
        chain_type_kwargs={"prompt": prompt },
        return_source_documents=True
        )

In [11]:
# Print statements for response details

def display_response_details(response):
    print("Query:", response["query"])
    print("Result:", response["result"])

    # Extract metadata from the first source document
    source_doc = response["source_documents"][0].metadata
    print("Source:", source_doc["source"])
    print("Description:", source_doc["description"])
    print("Title:", source_doc["title"])

In [14]:
# Set the question
question = "What can you tell my about QwQ model?"

# Initialize the query
response = qa_chain.invoke({"query": question, "context": retriever})

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.73it/s, est. speed input: 1825.16 toks/s, output: 152.38 toks/s]


In [15]:
# Example usage (replace 'response' with the actual response dictionary):
display_response_details(response)

Query: What can you tell my about QwQ model?
Result:  The QwQ model (Qwen with Questions) shows that it can think, question, and understand in a deep way. It is like a student who always wonders and thinks carefully before giving an answer. Just like learning new things, it knows that it knows nothing and keeps asking questions to learn more. It also has some limitations and needs to learn more as it goes. Just like everyone, it's always growing and improving.
Source: https://qwenlm.github.io/blog/qwq-32b-preview/
Description: GITHUB HUGGING FACE MODELSCOPE DEMO DISCORD
Note: This is the pronunciation of QwQ: /kwju:/ , similar to the word “quill”.
What does it mean to think, to question, to understand? These are the deep waters that QwQ (Qwen with Questions) wades into. Like an eternal student of wisdom, it approaches every problem - be it mathematics, code, or knowledge of our world - with genuine wonder and doubt. QwQ embodies that ancient philosophical spirit: it knows that it knows

In [14]:
# Set the question
question = "What can you tell my about Qwen2.5 open-source models: Qwen2.5, Qwen2.5-Coder, Qwen2.5-Math?"

# Initialize the query
response = qa_chain.invoke({"query": question, "context": retriever})

Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.94s/it, est. speed input: 611.53 toks/s, output: 165.20 toks/s]


In [15]:
# Example usage (replace 'response' with the actual response dictionary):
display_response_details(response)

Query: What can you tell my about Qwen2.5 open-source models: Qwen2.5, Qwen2.5-Coder, Qwen2.5-Math?
Result:  The Qwen2.5 models, namely Qwen2.5, Qwen2.5-Coder, and Qwen2.5-Math, have acquired significant improvements in various aspects compared to their predecessors. Here are some key takeaways:

1. **Academic Performance:**
   - Qwen2.5 achieved the best performance among open-source models on multiple popular code generation benchmarks (EvalPlus, LiveCodeBench, BigCodeBench).
   - Qwen2.5 also scored highly on an Instructive evaluation task, indicating strong alignment with human-like instruction-following abilities.
   - Qwen2.5 provided competitive performance with GPT-4o, demonstrating superiority over its predecessors.

2. **Practical Applications:**
   - Qwen2.5 models excel in handling different programming languages, showcasing their adaptability and versatility.
   - Qwen2.5 models have undergone significant enhancements, extending their skills to handle more complex reasonin